In [1]:
import pandas as pd
from keras.layers.core import Dense, Dropout
from keras.layers.recurrent import GRU
from keras.models import Sequential, load_model
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import copy


In [ ]:
import keras
import pandas as pd
from keras.layers.core import Dense, Dropout
from keras.layers.recurrent import GRU
from keras.models import Sequential, load_model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import copy

from keras.layers.recurrent import GRU
from keras.models import Model
from keras.layers import Input, Convolution2D, Flatten, Dense, Concatenate,Dropout
import copy
import numpy as np

true_stand_data=np.load(r'.//data3//true_array.npy')
false_stand_data=np.load(r'.//data3//false_array.npy')

sample_num={'true_num':true_stand_data.shape[0],'false_num':false_stand_data.shape[0]}
class_weight = {
                0: (1 / sample_num['false_num'] * (sample_num['true_num'] + sample_num['false_num'])) / 2,
                1: (1 / sample_num['true_num'] * (sample_num['true_num'] + sample_num['false_num'])) / 2
                }


true_sample_channel_1 = []
true_sample_channel_2 = []


false_sample_channel_1 = []
false_sample_channel_2 = []

for sample_number in range(true_stand_data.shape[0]):
    true_sample_channel_1.append(copy.deepcopy(true_stand_data[sample_number,:,:,0]))
#     true_sample_channel_2.append(copy.deepcopy(true_stand_data[sample_number,:,:,1]))

for sample_number in range(false_stand_data.shape[0]):
    false_sample_channel_1.append(copy.deepcopy(false_stand_data[sample_number,:,:,0]))
#     false_sample_channel_2.append(copy.deepcopy(false_stand_data[sample_number,:,:,1]))

combin_train_data1 = np.concatenate((true_sample_channel_1,false_sample_channel_1),axis=0)
# combin_train_data2 = np.concatenate((true_sample_channel_2,false_sample_channel_2),axis=0)

combin_target_data= np.concatenate((np.ones((sample_num['true_num'],1)),np.zeros((sample_num['false_num'],1))),axis=0)
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder()
combin_target_data=ohe.fit_transform(combin_target_data).toarray()

# Model 1
in1 = Input(shape=(110,7))
x_7_left = GRU(units=7,input_shape=(110,21))(in1)
x_21_left = GRU(units=21,input_shape=(110,21))(in1)
x_64_left = GRU(units=64,input_shape=(110,21))(in1)
x_192_left = GRU(units=192,input_shape=(110,21))(in1)
x_576_left = GRU(units=576,input_shape=(110,21))(in1)
x_1024_left = GRU(units=1024,input_shape=(110,21))(in1)

x_left_concat = Concatenate(axis=-1)([x_7_left,x_21_left,x_64_left,x_192_left,x_576_left,x_1024_left])

model_final_dense_1 = Dense(100, activation='relu')(x_left_concat)
model_final_dense_1 = Dropout(0.2)(model_final_dense_1)
model_final_dense_out = Dense(2, activation='softmax')(model_final_dense_1)


model = Model(inputs=[in1], outputs=model_final_dense_out)
print(model.summary())

import os
if 'GRU_predict.h5' in os.listdir('./'):
    model = load_model("./GRU_predict.h5")

callbacks_list = [
    
    keras.callbacks.EarlyStopping(
        monitor='accuracy', 
        patience=3 
    ),
    # 保存模型
    keras.callbacks.ModelCheckpoint(
    filepath = 'GRU_predict.h5', 
    monitor='loss', 
    save_best_only=True) 
]


model.compile(loss='categorical_crossentropy', #continu together
#               optimizer='adam',
              optimizer=tf.keras.optimizers.SGD(learning_rate=0.001),
              metrics=['accuracy']
             
             )

model.fit(x=combin_train_data1,
          y=combin_target_data,
          batch_size=120, 
          epochs=100,
          shuffle=True,
          verbose=1,
          class_weight=class_weight,
          callbacks=callbacks_list      
         )
